## Ingest circuits.csv file

1. Read the file
    1. Create an schema for the dataframe
2. Transform the DataFrame
    1. Select only wanted columns
    2. Rename columns to keep Data Standards
    3. Add Ingestion Date
3. Load Data to Silver in Parquet Format

In [66]:
from pyspark.sql import SparkSession
import os

### GET FILE PATH (LOCALLY)
cwd = os.getcwd()
circuits_path = cwd + '/bronze/circuits.csv'

In [67]:
### START SPARK SESION

spark = SparkSession.builder.appName('f1Practice').getOrCreate()

In [68]:
### CREATE DF SCHEMA
from pyspark.sql.types import IntegerType, StringType, DoubleType, StructType, StructField

schema = StructType(fields=[StructField('circuitId', IntegerType(), True),
                            StructField('circuitRef', StringType(), True),
                            StructField('name', StringType(), True),
                            StructField('location', StringType(), True),
                            StructField('country', StringType(), True),
                            StructField('lat', DoubleType(), True), 
                            StructField('lng', DoubleType(), True),
                            StructField('alt', IntegerType(), True),
                            StructField('url', StringType(), True)
                            ])


In [69]:
### LOAD CSV FILE INTO DATA FRAME

df = spark.read.option('header', True).schema(schema).csv(circuits_path)

In [70]:
### SELECT ONLY WANTED COLUMNS
from pyspark.sql.functions import col

### Option 1
# df_selected = df.select('circuitId', 'circuitRef', 'name', 'location', 'country', 'lat', 'lng', 'alt')

### Option 2
# df_selected = df.select(df.circuitId, df.circuitRef, df.name, df.location, df.country, df.lat, df.lng, df.alt) 

### Option 3 Allows you to apply additional Operations
selected_df = df.select(col('circuitId'),  ### You can .alias('new_name')
                        col('circuitRef'),
                        col('name'),
                        col('location'),
                        col('country'),
                        col('lat'),
                        col('lng'),
                        col('alt')) 

In [71]:
### RENAME COLUMNS

renamed_df = selected_df.withColumnRenamed('circuitId', 'circuit_id') \
                        .withColumnRenamed('circuitRef', 'circuit_ref') \
                        .withColumnRenamed('lat', 'latitude') \
                        .withColumnRenamed('lng', 'longitude') \
                        .withColumnRenamed('alt', 'altitude')


In [72]:
# ### ADD NEW COLUMN FOR INGESTION DATE
from pyspark.sql.functions import current_timestamp

final_df = renamed_df.withColumn('ingestion_date', current_timestamp())

In [73]:
### LOAD DATA TO SILVER FOLDER IN PARQUET FORMAT

silver_circuit_path = '/silver/circuits'
silver_path = cwd + silver_circuit_path

final_df.write.mode('overwrite').parquet(silver_path)
